In [13]:
import os
import glob
import re
from datetime import datetime
import whisper
import warnings
import textwrap
from dotenv import load_dotenv
import openai
from openai import OpenAI
import json

### Setup

In [2]:
# Suppress specific warnings
warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")

In [3]:
# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=os.getenv('OPENAI_API_KEY'),
)

In [4]:
# Load the Whisper model once before processing
model_size = 'large'  # You can set this to 'large' or any other size
print(f"Loading Whisper model '{model_size}'...")
model = whisper.load_model(model_size)
print("Model loaded successfully.")

Loading Whisper model 'large'...


/home/mparoca/anaconda3/envs/doctorvoiceai/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, ma

Model loaded successfully.


In [5]:
# Directory containing your audio files
audio_dir = 'data/test_audio'

# Get a list of all audio files in the directory (assuming .ogg format)
audio_files = glob.glob(os.path.join(audio_dir, '*.ogg'))

### Functions

In [6]:
# Function to extract date and time
def extract_datetime_from_filename(file_name):
    """
    Extracts the recording date and time from a WhatsApp audio file name.

    Parameters:
        file_name (str): The name of the audio file.

    Returns:
        datetime.datetime: The extracted recording date and time.
        None: If the date and time could not be extracted.
    """
    # Regular expression pattern to match the date and time in the file name
    pattern = r'WhatsApp Ptt (\d{4}-\d{2}-\d{2}) at (\d{1,2}\.\d{2}\.\d{2} [APM]{2})'
    match = re.search(pattern, file_name)

    if match:
        date_str = match.group(1)
        time_str = match.group(2)

        # Combine date and time strings
        datetime_str = f"{date_str} {time_str}"
        try:
            # Parse the datetime string into a datetime object
            recording_datetime = datetime.strptime(datetime_str, '%Y-%m-%d %I.%M.%S %p')
            return recording_datetime
        except ValueError as e:
            print(f"Error parsing date and time: {e}")
            return None
    else:
        print("Date and time not found in file name.")
        return None


In [7]:
# Function to transcribe audio (note that 'model' is now a parameter)
def transcribe_audio(file_path, model, language='es'):
    """
    Transcribes an audio file using a pre-loaded Whisper model.

    Parameters:
        file_path (str): The path to the audio file.
        model: The pre-loaded Whisper model.
        language (str): The language code for the audio.

    Returns:
        str: The transcribed text.
    """
    try:
        # Transcribe the audio file
        result = model.transcribe(file_path, language=language)
        return result['text']
    except Exception as e:
        print(f"An error occurred during transcription: {e}")
        return None

In [8]:
# Function to do an additional pass to correct medical terminology
def correct_medical_text(transcribed_text):
    """
    Uses OpenAI's API to correct medical terminology in the transcribed text.

    Parameters:
        transcribed_text (str): The text to be corrected.

    Returns:
        str: The corrected text.
    """
    try:
        # Define the conversation messages
        messages = [
            {
                "role": "system",
                "content": (
                    "Eres un asistente médico altamente capacitado que corrige transcripciones de notas clínicas en español. "
                    "Tu tarea es revisar el siguiente texto transcrito, corregir errores gramaticales y de ortografía, "
                    "asegurarte de que la terminología médica sea precisa y estandarizada, y convertir las unidades de medida "
                    "escritas en palabras a sus abreviaturas estándar. Por ejemplo, convierte 'miligramos por decilitro' a 'mg/dL', "
                    "'mililitros por minuto' a 'mL/min', 'gramos por decilitro' a 'g/dL', 'meq por litro' a 'mEq/L', "
                    "entre otras conversiones similares. Mantén la claridad y la profesionalidad en el texto corregido."
                )
            },
            {
                "role": "user",
                "content": transcribed_text
            }
        ]

        # Create the chat completion
        response = client.chat.completions.create(
            model='gpt-4',  # Use 'gpt-3.5-turbo' if you don't have access to 'gpt-4'
            messages=messages,
            max_tokens=1024,
            temperature=0,  # Set to 0 for deterministic output
        )

        # Correct way to access the response content using attribute access
        corrected_text = response.choices[0].message.content.strip()
        return corrected_text

    except Exception as e:
        print(f"An error occurred while correcting the text: {e}")
        return transcribed_text  # Return the original text if correction fails

In [14]:
def extract_insights_openai(corrected_text):
    """
    Extracts key medical insights from the corrected text using OpenAI's ChatCompletion API.

    Parameters:
        corrected_text (str): The corrected medical text.

    Returns:
        dict: A dictionary containing extracted insights.
    """
    try:
        # Define the system and user messages
        messages = [
            {
                "role": "system",
                "content": (
                    "Eres un asistente médico experto. Tu tarea es extraer información clave de la siguiente nota clínica. "
                    "Organiza la información en las siguientes categorías: "
                    "1. Información del Paciente, "
                    "2. Diagnóstico, "
                    "3. Resultados de Laboratorio, "
                    "4. Tratamientos Actuales, "
                    "5. Recomendaciones. "
                    "Para cada categoría, proporciona los detalles relevantes. "
                    "Usa el siguiente formato JSON sin explicaciones adicionales:"
                )
            },
            {
                "role": "user",
                "content": corrected_text
            }
        ]

        # Create the chat completion
        response = client.chat.completions.create(
            model='gpt-4',  # Use 'gpt-3.5-turbo' if you don't have access to 'gpt-4'
            messages=messages,
            max_tokens=1024,
            temperature=0,  # Set to 0 for deterministic output
        )

        # Access the response content using attribute access
        json_response = response.choices[0].message.content.strip()

        # Parse the JSON response
        insights = json.loads(json_response)
        return insights

    except json.JSONDecodeError:
        print("Error decoding JSON response from OpenAI.")
        return {}
    except Exception as e:
        print(f"An error occurred while extracting insights: {e}")
        return {}

### Test

In [9]:
# Test the function on each audio file
for file_path in audio_files:
    # Extract the file name from the path
    file_name = os.path.basename(file_path)
    print(f"\nProcessing file: {file_name}")

    # Call the function to extract date and time
    recording_datetime = extract_datetime_from_filename(file_name)

    # Print the result
    if recording_datetime:
        print("Recording Date and Time:", recording_datetime)
    else:
        print("Failed to extract date and time from the file name.")


Processing file: WhatsApp Ptt 2024-09-22 at 12.19.53 AM.ogg
Recording Date and Time: 2024-09-22 00:19:53


In [10]:
# Test the function on each audio file
for file_path in audio_files:
    # Extract the file name from the path
    file_name = os.path.basename(file_path)
    print(f"\nProcessing file: {file_name}")

    # Transcribe the audio (pass the pre-loaded model)
    transcribed_text = transcribe_audio(file_path, model)

    # Print the result
    if transcribed_text:
        print("Transcribed Text:")
        # Wrap the entire text for better readability
        wrapped_text = textwrap.fill(transcribed_text, width=80)
        print(wrapped_text)

        print("\n" + "=" * 40 + "\n")  # Separator for better readability
    else:
        print("Failed to transcribe the audio.")



Processing file: WhatsApp Ptt 2024-09-22 at 12.19.53 AM.ogg
Transcribed Text:
 Equipo, quería comentarles sobre el paciente anónimo Pérez, hombre de 60 años
con diagnóstico de enfermedad renal crónica estadio 3. En los últimos análisis
de laboratorio su creatinina sérica está en 2.4 miligramos decilitros y el
filtrado glomerular estimado es de 35 mililitros min. También presenta
proteinuria en rango moderado. Además, los niveles de potasio están ligeramente
elevados en 5.3 MeKL y tiene anemia normocrítica con hemoglobina en 10 gramos
decilitros. Ha estado considerando ajustar su tratamiento antihipertensivo y
evaluar la necesidad de iniciar eritropoyetina para manejar la anemia. Me
gustaría conocer sus opiniones sobre estos ajustes y si creen pertinente
derivarlo a nefrología para una evaluación más detallada. Revisen el caso y
compartan sugerencias. Gracias.




In [11]:
# Proceed if transcription was successful
if transcribed_text:
    #print("Original Transcribed Text:")
    wrapped_original = textwrap.fill(transcribed_text, width=80)
    #print(wrapped_original)

    # Correct the transcribed text using OpenAI
    corrected_text = correct_medical_text(transcribed_text)

    print("\nCorrected Text:")
    wrapped_corrected = textwrap.fill(corrected_text, width=80)
    print(wrapped_corrected)

    print("\n" + "=" * 40 + "\n")  # Separator for better readability
else:
    print("Failed to transcribe the audio.")



Corrected Text:
Equipo, quería comentarles sobre el paciente anónimo Pérez, hombre de 60 años
con diagnóstico de enfermedad renal crónica estadio 3. En los últimos análisis
de laboratorio, su creatinina sérica está en 2.4 mg/dL y el filtrado glomerular
estimado es de 35 mL/min. También presenta proteinuria en rango moderado.
Además, los niveles de potasio están ligeramente elevados en 5.3 mEq/L y tiene
anemia normocrítica con hemoglobina en 10 g/dL. Ha estado considerando ajustar
su tratamiento antihipertensivo y evaluar la necesidad de iniciar eritropoyetina
para manejar la anemia. Me gustaría conocer sus opiniones sobre estos ajustes y
si creen pertinente derivarlo a nefrología para una evaluación más detallada.
Revisen el caso y compartan sugerencias. Gracias.




In [15]:
# Extract insights
insights = extract_insights_openai(corrected_text)

# Pretty-print the extracted insights
print("\nExtracted Insights:")
print(json.dumps(insights, indent=4, ensure_ascii=False))


Extracted Insights:
{
    "Información del Paciente": {
        "Nombre": "Pérez",
        "Edad": "60 años",
        "Sexo": "Hombre"
    },
    "Diagnóstico": {
        "Principal": "Enfermedad renal crónica estadio 3",
        "Secundario": [
            "Proteinuria en rango moderado",
            "Anemia normocrítica",
            "Niveles de potasio ligeramente elevados"
        ]
    },
    "Resultados de Laboratorio": {
        "Creatinina sérica": "2.4 mg/dL",
        "Filtrado glomerular estimado": "35 mL/min",
        "Niveles de potasio": "5.3 mEq/L",
        "Hemoglobina": "10 g/dL"
    },
    "Tratamientos Actuales": {
        "Antihipertensivo": "En revisión para ajuste"
    },
    "Recomendaciones": {
        "Tratamiento": [
            "Evaluar la necesidad de iniciar eritropoyetina para manejar la anemia"
        ],
        "Derivación": [
            "Considerar derivación a nefrología para una evaluación más detallada"
        ]
    }
}
